# ÖPNV Schkeuditz
fetched the data with this overpass query:
[https://overpass-turbo.eu/s/1P2i](https://overpass-turbo.eu/s/1P2i)
```json overpass
[out:json][timeout:25];
// gather results
(
  nwr["route"="bus"]["network"!="Flixbus"]["network"!="Eurolines"]({{bbox}});
  nwr["railway"="station"]({{bbox}});
  nwr["railway"="rail"]({{bbox}});
  nwr["railway"="tram"]({{bbox}});
  nwr["railway"="platform"]({{bbox}});
);
// print results
out geom;
```

In [ ]:
import pandas as pd
import numpy as np
import folium
import osmnx as ox
import pyproj # we need this for jupyter-book to build right
import geopandas as gpd

In [ ]:
# Autonomer Bus
# Hayna, Biedermeierstrand 6570009526
# Rackwitz, Schladitzer Bucht 10801206725
# Neuschladitz 262240958
# Rackwitz, Leipziger Straße/Hauptstraße 294771252
# Rackwitz, Busbahnhof 2674713925

In [ ]:
# Read the GeoJSON file
gdf = gpd.read_file(
    r'C:\Users\plank\Documents\git\r4r-prototypes\prototypes\in\a\001.geojson')

gdf.head()

In [ ]:
for entry in gdf.columns:
    print(entry)

In [ ]:
# Create a map
m = folium.Map(location=[gdf.centroid.y.mean(),
               gdf.centroid.x.mean()], zoom_start=12, tiles="cartodbpositron")

In [ ]:
def get_coordinates_from_geometry(geometry):
    coordinates = list(geometry.coords)
    return [(lon, lat) for lat, lon in coordinates]

In [ ]:
def plot_geometry(m, row, color):
    geometry = row.geometry
    if geometry.geom_type == 'MultiLineString':
        for part in geometry.geoms:
            coordinates = get_coordinates_from_geometry(part)
            polyline = folium.PolyLine(
                locations=coordinates, color=color)
            # Add the polyline to the map
            polyline.add_to(m)
    elif geometry.geom_type == 'Polygon':
        coordinates = get_coordinates_from_geometry(geometry.exterior)
        polygon = folium.Polygon(
            locations=coordinates, color=color, fill=True, fill_color=color, fill_opacity=0.4)
        # Add the polygon to the map
        polygon.add_to(m)
    elif geometry.geom_type == 'LineString':
        coordinates = get_coordinates_from_geometry(geometry)
        polyline = folium.PolyLine(
            locations=coordinates, color=color)
        polyline.add_to(m)
    elif geometry.geom_type == 'Point':
        coordinates = get_coordinates_from_geometry(geometry)
        # print(coordinates)
        # marker = folium.Marker(location=coordinates[0], color=color)
        marker = folium.CircleMarker(location=coordinates[0], color=color)
        marker.add_to(m)

    else:
        print(geometry.geom_type)
        coordinates = get_coordinates_from_geometry(geometry)
        polyline = folium.PolyLine(
            locations=coordinates, color=color)
        polyline.add_to(m)

In [ ]:
# make layer
layer_bus = folium.FeatureGroup(name='Bus')
layer_flash = folium.FeatureGroup(name='Flash')
# layer_tram = folium.FeatureGroup(name='Tram')
layer_train = folium.FeatureGroup(name='Train')
layer_airport = folium.FeatureGroup(name='Airport')
layer_bike = folium.FeatureGroup(name='Bike')

m.add_child(layer_bus)
m.add_child(layer_flash)
# m.add_child(layer_tram)
m.add_child(layer_train)
m.add_child(layer_airport)
m.add_child(layer_bike)

m.add_child(folium.LayerControl())

In [ ]:
color = "black"
for row in gdf.itertuples():
    # Create a string for popup
    if row.route == "bus":
        if row.id == "relation/15706522":  # Das ist Flash https://www.openstreetmap.org/relation/15706522#map=15/51.4358/12.3750
            color = "orange"
            plot_geometry(layer_flash, row, color)
            continue
        color = "blue"
        plot_geometry(layer_bus, row, color)
    elif row.railway is not None:
        if row.railway in ["signal", "switch", "buffer_stop", "crossing", "tram_level_crossing", "level_crossing"]:
            continue
        elif row.railway in ["station", "rail", "tram", "platform"]:
            color = "red"
            plot_geometry(layer_train, row, color)
    else:
        x = eval(row._158)[0]
        # print(x)
        # for entry in x:
        #     print(entry)
        if x.get("role") in ["stop", "platform"]:
            if x.get("reltags").get("route") == "bus":
                color = "blue"
                layer = layer_bus
            else:
                print(x.get("reltags").get("route"))
                color = "red"
                layer = layer_train
            plot_geometry(layer, row, color)

In [ ]:
# Flughafen
way_id = 435715317
boundary = ox.geocode_to_gdf("W435715317", by_osmid=True)
boundary.exterior[0].coords

In [ ]:
coordinates = list(boundary.exterior[0].coords)
reversed_coordinates = [(lon, lat) for lat, lon in coordinates]
polygon = folium.Polygon(
    locations=reversed_coordinates, color="green", fill=True, fill_color="green", fill_opacity=0.1)
# Add the polygon to the map
polygon.add_to(layer_airport)

In [ ]:
m

In [ ]:
m.save("out/Schkeuditz-Europa.html")